<a href="https://colab.research.google.com/github/MarcelloCeresini/ChessBreaker/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
username = 'MarcelloCeresini'
repository = 'ChessBreaker'

In [ ]:
# COLAB ONLY CELLS
try:
    import google.colab
    IN_COLAB = True
    !nvidia-smi             # Check which GPU has been chosen for us
    !rm -rf logs
    #from google.colab import drive
    #drive.mount('/content/drive')
    #%cd /content/drive/MyDrive/GitHub/
    !git clone https://github.com/{username}/{repository}.git
    %cd {repository}
    %ls
    !pip3 install anytree
except:
    IN_COLAB = False

In [ ]:
import numpy as np
import tensorflow as tf
import chess
from anytree import Node
from time import time
import matplotlib.pyplot as plt
from tqdm import tqdm
from queue import Queue
import ray

ray.init()

import utils
from utils import plane_dict, Config, x_y_from_position
from model import ResNet

conf = Config()
board = chess.Board()

# legal_moves = board.legal_moves
# for move in legal_moves:
#     print(move.uci())  
# print(legal_moves)


In [ ]:
board = chess.Board()
board.set_fen('rnbqkbnr/p1pppppp/8/1p6/4P3/8/PPPP1PPP/RNBQKBNR w KQkq')
board

In [ ]:
def special_input_planes(board):                                    # not repeated planes
    
    special_planes = np.zeros([*conf.BOARD_SHAPE, conf.SPECIAL_PLANES], conf.PLANES_DTYPE_NP)
    special_planes[:,:,0] = board.turn                                 
    special_planes[:,:,1] = board.fullmove_number-1                    
    special_planes[:,:,2] = board.has_kingside_castling_rights(True)   
    special_planes[:,:,3] = board.has_queenside_castling_rights(True)  
    special_planes[:,:,4] = board.has_kingside_castling_rights(False)  
    special_planes[:,:,5] = board.has_queenside_castling_rights(False) 
    special_planes[:,:,6] = board.halfmove_clock                       
    
    # special_planes = np.stack([
    #     np.full([*conf.BOARD_SHAPE], int(board.turn                                 ), conf.PLANES_DTYPE_NP),   # whose turn it is
    #     np.full([*conf.BOARD_SHAPE], int(board.fullmove_number-1                    ), conf.PLANES_DTYPE_NP),   # don't know why but it starts from 1 on move 1, just reduce it by one and now it's right (MAX 255, using uint8!!)
    #     np.full([*conf.BOARD_SHAPE], int(board.has_kingside_castling_rights(True)   ), conf.PLANES_DTYPE_NP),   # True for White
    #     np.full([*conf.BOARD_SHAPE], int(board.has_queenside_castling_rights(True)  ), conf.PLANES_DTYPE_NP),
    #     np.full([*conf.BOARD_SHAPE], int(board.has_kingside_castling_rights(False)  ), conf.PLANES_DTYPE_NP),   # False for Black
    #     np.full([*conf.BOARD_SHAPE], int(board.has_queenside_castling_rights(False) ), conf.PLANES_DTYPE_NP),
    #     np.full([*conf.BOARD_SHAPE], int(board.halfmove_clock                       ), conf.PLANES_DTYPE_NP)    # number of moves from last capture / pawn move --> reaching 50 means draw
    # ])

    return special_planes                                            # transpose to have plane number last --> in order to concat them


# def update_planes(current, board, board_history):

#     if current == None: # root, initialize to zero
#         current = tf.zeros([*conf.BOARD_SHAPE, conf.TOTAL_PLANES], dtype=conf.PLANES_DTYPE)
    
#     planes = [] # since we cannot "change" a tensor after creating it, we create them one by one in a list and then stack them

#     for color in range(2):                                                                                                  # for each color
#         for piece_type in range(1, conf.N_PIECE_TYPES+1):                                                                   # for each piece type
#             indices = []                                                                                                    # --> we save the position on the board in a list
#             for position in list(board.pieces(piece_type, color)):                                                          # for each piece of that type
#                 indices.append(x_y_from_position(position))                                                                 # the function transforms a number (1-64) into a tuple (1-8, 1-8)
#             if len(indices) == 0:
#                 tensor = uniform_tensor(tf.constant(0, dtype=conf.PLANES_DTYPE))
#             else:
#                 values = np.array([1]*len(indices), dtype=conf.PLANES_DTYPE_NP) # simply "1" in a list with unit8 dtype
#                 tensor = tf.sparse.to_dense(tf.SparseTensor(dense_shape=[*conf.BOARD_SHAPE], indices=indices, values=values))   ### created as sparse because it's easier, needed as dense afterwards
#             planes.append(tensor)
#         planes.append(uniform_tensor(tf.constant(board_history.count(board_history[-1]), dtype=conf.PLANES_DTYPE)))         # adding a "repetition plane" for each color (simply count how many times the current (last) position has been encountered)

#     # 1 stack
#     current_planes = tf.transpose(tf.stack(planes), [1,2,0])    
#     print(tf.shape(current_planes))                                                            # transpose them to have the planes as last dimension
#     # 7 stacks (total 8 repetitions)
#     old_planes = tf.slice(current, begin=[0,0,0], size=[*conf.BOARD_SHAPE, (conf.PAST_TIMESTEPS-1)*conf.REPEATED_PLANES])   # take the first 7 repetitions, slice them and paste them at the end of the new planes (last is discarded, as are special planes)
    
#     return tf.concat([current_planes, old_planes, special_input_planes(board)], axis=-1)    # also concat the special planes

def update_planes(old, board, board_history):

    if type(old) != np.ndarray: # root, initialize to zero
        old = np.zeros([*conf.BOARD_SHAPE, conf.TOTAL_PLANES], dtype=conf.PLANES_DTYPE_NP)
    
    total_planes = np.zeros([*conf.BOARD_SHAPE, conf.TOTAL_PLANES], dtype=conf.PLANES_DTYPE_NP) # since we cannot "change" a tensor after creating it, we create them one by one in a list and then stack them
    plane = -1
    
    repetition_counter = board_history.count(board_history[-1])
    for color in range(2):                                                                                                  # for each color
        for piece_type in range(1, conf.N_PIECE_TYPES+1):                                                                   # for each piece type
            plane += 1
            indices = map(x_y_from_position, list(board.pieces(piece_type, color)))        # for each piece of that type                                                                                            # --> we save the position on the board in a list
            # the function transforms a number (1-64) into a tuple (1-8, 1-8)
            for idx in indices:
                total_planes[idx[0], idx[1], plane] = 1
        plane += 1
        total_planes[:, :, plane] = repetition_counter    # adding a "repetition plane" for each color (simply count how many times the current (last) position has been encountered)
   
    # 7 stacks (total 8 repetitions)
    to_keep = tf.slice(old, begin=[0,0,0], size=[*conf.BOARD_SHAPE, (conf.PAST_TIMESTEPS-1)*conf.REPEATED_PLANES])   # take the first 7 repetitions, slice them and paste them at the end of the new planes (last is discarded, as are special planes)
    
    total_planes[:, :, conf.REPEATED_PLANES:(conf.REPEATED_PLANES+conf.OLD_PLANES_TO_KEEP)] = old[:, :, :conf.OLD_PLANES_TO_KEEP]
    total_planes[:, :, conf.REPEATED_PLANES+conf.OLD_PLANES_TO_KEEP:] = special_input_planes(board)
    
    return total_planes


#0.04062032699584961
# 0.0007264614105224609


In [ ]:
class MyNode(Node): # subclassing Node from Anytree to add some methods

    def update_action_value(self, new_action_value):                                                        # used during backtracking to update action value if the simulation reached the end through that node
        self.action_value += (new_action_value-self.action_value)/(self.visit_count+1)                      # simply the mean value, but computed iteratively

    def calculate_upper_confidence_bound(self, num_total_iterations=1):                                     # Q + U --> U proportional to P/(1+N) --> parameter decides exploration vs. exploitation
        new_UCF = self.action_value + conf.expl_param(num_total_iterations)*self.prior/(1+self.visit_count)
        return new_UCF

    def calculate_move_probability(self, num_total_iterations=1):                                           # N^(1/tau) --> tau is a temperature parameter (exploration vs. exploitation)
        return self.visit_count**(1/conf.temp_param(num_total_iterations))

@ray.remote
def expand_node(root_node, full_moves, outcome, legal_moves):
    # nodes_to_visit.put_nowait(root_node) ### remember
                        
    mask_idx = utils.mask_moves(legal_moves)
    priors = [full_moves[idx[0], idx[1], idx[2]] for idx in mask_idx]                        # boolean mask returns a tensor of only the values that were masked (as a list let's say)
    # 0.006434917449951172
    # 1.3113021850585938e-05
    root_node.action_value = outcome    

    for move, prior in zip(legal_moves, priors):                                                # creating children
        
        root_board_fen = root_node.board.fen()
        new_board = chess.Board()
        new_board.set_fen(root_board_fen)
        new_board.push(move)
                                    
        new_board_history = root_node.board_history.copy()                                      # and board history! (copy because list are pointers)
        new_board_history.append(new_board.fen()[:-6])

        planes = update_planes(root_node.planes, new_board, new_board_history)
        
        MyNode(
            move.uci(), 
            parent = root_node,                                                                 # very important to build the tree
            prior = prior,                                                                      # prior is the "initial" state_value of a node
            visit_count = 0,                                                                    # initialize visit_count to 0
            action_value = 0,
            board = new_board, 
            board_history = new_board_history,                                                  
            planes = planes             # update the planes --> each node stores its input planes!
        )
    
    return root_node 


def MTCS(model, root_node, max_depth, num_restarts):
    '''
        As it is written, the search descends until it finds a leaf to be evalued, then restarts until it gathers a batch of evaluations
        The descent, however, does not restart from the just evalued leafs, but always from the beginning (easier implementation)
        This should IMPROVE exploration because the probability of entering a node is inversely proportional to the number of visits of that node
    '''
    evaluation_counter = 0
    INIT_ROOT = root_node
    nodes_to_visit = Queue()
    # number of times to explore up until max_depth
    i = 0

    leaf_node_batch = []
    legal_moves_batch = []
    
    while i < num_restarts:
        if nodes_to_visit.empty():
            root_node = INIT_ROOT
            i+=1
        else:
            root_node = nodes_to_visit.get_nowait()
        # print(i, root_node.name, root_node.depth, root_node.visit_count)
        
        while root_node.depth <= max_depth:  # while depth < max --> descend BUT STOP when you finished num_restarts AND the batch is empty AND you reach an already explored end of the tree with a subsequent descent
            
            # TODO: implement "complete game" inside MTCS --> stop the descent and give 1 as outcome (1/-1 depends)
            
            assert root_node.depth >= 0 and root_node.depth <= max_depth, "depth is wrong"          

            if root_node.is_leaf:                                                                           # if it's leaf --> need to pass the position (planes) through the model, to get priors (action_values) and outcome (state_value)
                if len(root_node.siblings) > 0:         # this part is to try and avoid batching the same node twice (so we evaluate a random sibling instead)
                    leaf_node_name_list = [node.name for node in leaf_node_batch]
                    if root_node.name in leaf_node_name_list:
                        siblings_list = list(root_node.siblings)
                        random_sibling = np.random.choice(siblings_list)
                        while random_sibling.name in leaf_node_name_list and len(siblings_list)>1:
                            siblings_list.remove(random_sibling) #do it with np.random in a range, and POP instead of remove
                            random_sibling = np.random.choice(siblings_list)
                        
                        root_node = random_sibling
                        
                # important! save legal moves AFTER choosing root_node
                legal_moves = list(root_node.board.legal_moves)
                leaf_node_batch.append(root_node)
                legal_moves_batch.append(legal_moves)
                root_node.visit_count += 1

                # print("to be evaluated", "d", root_node.depth, "vc", root_node.visit_count, "name", root_node.name)

                if len(leaf_node_batch) == conf.BATCH_DIM or root_node == INIT_ROOT:


                    plane_list = [root_node.planes for root_node in leaf_node_batch]
                    # 0.0032072067260742188
                    # 7.05718994140625e-05
                    
                    planes = np.stack(plane_list)

                    full_moves_batch, outcome_batch = model(planes)
                    evaluation_counter+=1

                    full_moves_batch_np = full_moves_batch.numpy()
                    outcome_batch_np = outcome_batch.numpy()

                    def unstack(a, axis=0):
                        return np.moveaxis(a, axis, 0)

                    if np.shape(full_moves_batch_np)[0] != 1:
                        full_moves_batch_np = unstack(full_moves_batch.numpy())
                        outcome_batch_np = unstack(outcome_batch.numpy())

                    #########################
                    futures = [
                        expand_node(root_node, full_moves, outcome, legal_moves)
                        for root_node, full_moves, outcome, legal_moves 
                        in zip(leaf_node_batch, full_moves_batch_np, outcome_batch_np, legal_moves_batch)
                    ]
                    
                    nodes_batch = ray.get(futures)
                    [nodes_to_visit.put_nowait(node) for node in nodes_batch]
                            
                    leaf_node_batch = []
                    legal_moves_batch = []
                    break # restart another iteration

                else:
                    # after
                    root_node = INIT_ROOT
                    i += 1

            else: # if it does not need to be evalued because it already has children 
                if root_node.depth < max_depth:                                                                 # if we are normally descending
                    # print("choosing point", "d", root_node.depth, "vc", root_node.visit_count, "name", root_node.name)
                    children = root_node.children                                                               # get all the children (always != [])
                    
                    values = [child.calculate_upper_confidence_bound() for child in children]
                    root_node = children[np.argmax(values)]
                    root_node.visit_count += 1                                                                  # add 1 to the visit count of the chosen child
                    # print("chosen node", "d", root_node.depth, "vc", root_node.visit_count, "name", root_node.name)
                else:
                    # print("final leaf", "d", root_node.depth, "vc", root_node.visit_count, "name", root_node.name, root_node.calculate_upper_confidence_bound())
                    flag_max_depth = False
                    outcome = root_node.action_value    # needed for when depth=max_depth AND NOT LEAF (that means, already visited leaf) --> don't REDO the evaluation, it would give the same result, simply copy it from before
                    break                               # it will leave the while, max depth is reached
           
        # print("num_restarts", i, " - flag_r", flag_restart, " - flag_md", flag_max_depth)
        # barckpropagation of action value through the tree
        # print("outcome", outcome)
        while root_node.depth > 0:
            # root node should be an already evalued leaf, at max depth (so OUTCOME has been set)
            assert root_node.depth > 0 and root_node.depth <= max_depth, "depth is wrong"
            root_node = root_node.parent
            root_node.update_action_value(outcome)

    return INIT_ROOT, evaluation_counter


def choose_move(root_node):
    children = root_node.children
    assert root_node.children != [], "No children, cannot choose move"
    p = [child.calculate_move_probability() for child in children] 
    p_norm = [i/sum(p) for i in p] # normalize probabilities
    root_node = np.random.choice(
        children, 
        p = p_norm  # choose the child proportionally to the number of times it has been visited (exponentiated by a temperature parameter)
    ) 
    root_node.parent = None # To detach the subtree and restart with the next move search

    return root_node


def complete_game(model, max_depth=7, num_restarts=100):
    move_list = []
    board = chess.Board()
    board_history = [board.fen()[:-6]]                           # we remove the "en passant", "halfmove clock" and "fullmove number" from the fen --> position will be identical even if those values differ
    root_node = MyNode(
        "Start",                                                     # no name needed for initial position
        board = board,
        board_history = board_history,
        planes = update_planes(None, board, board_history),    # start from empty planes and fill them (usually you need previous planes to fill them)
        action_value=0,
        visit_count=0
        )

    results = []
    # while not root_node.board.is_game_over(claim_draw=True) and root_node.board.fullmove_number <= conf.MAX_MOVE_COUNT:
    while not root_node.board.is_game_over(claim_draw=True):
        tic = time()
        root_node, eval_c = MTCS(model, root_node, max_depth = max_depth, num_restarts=num_restarts)                          # though the root node you can access all the tree
        root_node = choose_move(root_node)
        move_list.append(chess.Move.from_uci(root_node.name))
        results.append((
            int(root_node.planes[0,0,conf.REPEATED_PLANES+conf.OLD_PLANES_TO_KEEP+1]),  # it is the fullmove number
            root_node.visit_count,
            eval_c,
            time()-tic,
            root_node.action_value))

    return move_list, root_node.board.outcome(claim_draw=True), results


In [ ]:
model = ResNet()
print(tf.shape(conf.DUMMY_INPUT))
fm, ac = model(conf.DUMMY_INPUT)
print("full moves shape", fm.shape)
print("action values shape", ac.shape)
model.summary()

In [ ]:
moves, outcome, results = complete_game(model, max_depth=3, num_restarts=15)


In [ ]:
# res_dict = {}
# games = []
# outcomes = []
# for depth in [4, 6, 8, 10]:
#     for n_rep in [100, 1000, 10000]:
#         tic = time()
#         moves, outcome, results = complete_game(model, max_depth=depth, num_restarts=n_rep)
#         res_dict[(depth, n_rep, "eval")] = np.average([res[2] for res in results])
#         res_dict[(depth, n_rep, "time")] = np.average([res[3] for res in results])
#         games.append(moves)
#         outcomes.append(outcome)
#         print(depth, n_rep, time()-tic)

In [ ]:
# outcome

In [ ]:
# moves2 = moves.copy()
# board = chess.Board()

In [ ]:
# for i in range(280):
#     board.push(moves2.pop(0))

In [ ]:
# board.push(moves2.pop(0))
# board